# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Capstone Project

Notebook 2: Data Cleaning & EDA

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [10]:
script = pd.read_csv('../data/seinfeld_scripts_no_title.csv')
script.head()

,character,line
0,JERRY,"You know, why we're here? To be out, this is out...and out is one of the single most enjoyable experiences of life. People...did you ever hear people talking about ""We should go out""? This is what they're talking about...this whole thing, we're all out now, no one is home. Not one person here is home, we're all out! There are people tryin' to find us, they don't know where we are. (imitates one of these people ""tryin' to find us""; pretends his hand is a phone) ""Did you ring?, I can't find him."" (imitates other person on phone) ""Where did he go?"" (the first person again) ""He didn't tell me where he was going"". He must have gone out. You wanna go out: you get ready, you pick out the clothes, right? You take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...There you're staring around, whatta you do? You go: ""We gotta be getting back"". Once you're out, you wanna get back! You wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? Where ever you are in life, it's my feeling, you've gotta go. (Pete's luncheonette. Jerry and George are sitting at a table.)"
1,JERRY,"Seems to me, that button is in the worst possible spot. (talking about George's shirt) The second button literally makes or breaks the shirt, look at it: it's too high! It's in no-man's-land, you look like you live with your mother."
2,GEORGE,Are you through? (kind of irritated)
3,JERRY,"You do of course try on, when you buy?"
4,GEORGE,"Yes, it was purple, I liked it, I don't actually recall considering the buttons."
